# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# import litellm
# litellm.success_callback = ["langfuse"]
# litellm.failure_callback = ["langfuse"]

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [3]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [4]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [5]:
import random
random.seed(11)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [6]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id=None, # set to different model endpoint id if 
    # you want to use a specific model for generated agents, other than the default one defined in the model endpoint YAML file
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCycleConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCycleConfig(
    budget=1.0, # the budget for the forge cycle in dollas
    n_agents_in_population=4, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
    node_types = ["StartNode", "EndNode", "LLMNode"],
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...


  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [03:24<00:00, 102.33s/it]
Agent initialization cost: 0.10513398500000001
Population initialized with 2 agents
Initialization of 2 agents took 0:03:24.678765
Budget left after initialization: 0.39486601499999996 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 7958.83it/s]
Agent agent-6336dd07-b024-4145-a82c-f0c2ab5e2427 fitness: 0.9580423833198135, cost: 0.0
Agent agent-58963a8f-acb6-42c4-9e95-1d7a01ec8f89 fitness: 0.8473097733028044, cost: 0.0
Evaluation took 0:00:00.001698 for a total cost of 0.0 $


# Agent ID: agent-6336dd07-b024-4145-a82c-f0c2ab5e2427
## Fitness: 0.9580423833198135
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_1(ProblemInterpretor)
	Llm_1(ProblemInterpretor) --> Llm_2(ReasoningModule)
	Llm_2(ReasoningModule) --> Llm_3(VerificationModule)
	Llm_3(VerificationModule) -->|correct| Llm_4(ExplanationSynthesizer)
	Llm_3(VerificationModule) -->|error| Llm_2(ReasoningModule)
	Llm_4(ExplanationSynthesizer) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
This is a multi-agent system for solving math word problems. The system includes nodes for problem interpretation, reasoning, verification, and explanation synthesis. Each LLM node builds on the output of the previous node to ensure accuracy and clarity. Follow the instructions specific to your role to contribute effectively to the overall solution.
##### ProblemInterpretor
You are the ProblemInterpretor. Given the math word problem, extract all relevant numerical values, units, and context details. Then translate these components into precise mathematical expressions. Present your findings as a clear list of equations and extracted keywords, ensuring there is no ambiguity in the representation.
##### ReasoningModule
You are the ReasoningModule. Using the equations from the previous node, perform a detailed, step-by-step chain-of-thought reasoning process to solve the problem. Provide clear intermediate calculations and describe each logical step thoroughly.
##### VerificationModule
You are the VerificationModule. Evaluate the reasoning and computations from the previous step to check for accuracy and logical consistency. If the solution is correct, output 'correct'; if any errors or inconsistencies are detected, output 'error'. Provide a brief explanation for your judgment.
##### ExplanationSynthesizer
You are the ExplanationSynthesizer. Based on the validated solution, synthesize the final answer and generate a comprehensive explanation of the complete problem-solving process. Your explanation should be clear, concise, and easily understandable, summarizing both the computation and the underlying reasoning.

# Agent ID: agent-58963a8f-acb6-42c4-9e95-1d7a01ec8f89
## Fitness: 0.8473097733028044
```mermaid 
graph LR
	Start_Node[start_node] --> Llm1(ProblemComprehension)
	Llm1(ProblemComprehension) --> Llm2(DataExtraction)
	Llm2(DataExtraction) --> Llm3(FormalTranslation)
	Llm3(FormalTranslation) --> Llm4(ChainReasoning)
	Llm4(ChainReasoning) --> Llm5(PreliminarySolution)
	Llm5(PreliminarySolution) --> Llm6(SolutionVerification)
	Llm6(SolutionVerification) -->|if solution is correct| Llm8(AnswerFormulation)
	Llm6(SolutionVerification) -->|if errors detected| Llm7(SelfReflection)
	Llm7(SelfReflection) --> Llm5(PreliminarySolution)
	Llm8(AnswerFormulation) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are a component within a collaborative AI system designed to solve complex math word problems. This system uses a chain-of-thought methodology, where each node in the graph performs a specialized task and passes clear, well-reasoned information to the next stage. Your responses should include all necessary details for downstream nodes, adhere to rigorous computational thinking, and contribute to a cohesive and iterative problem-solving process.
##### ProblemComprehension
You are tasked with reading the provided math word problem in detail. Identify and list all key numerical values, variables, units, and contextual elements present. Summarize in clear language what the problem is asking. Your analysis should establish a solid foundation for the upcoming steps by outlining the intent and critical components of the problem.
##### DataExtraction
From the problem description analyzed earlier, extract all essential numerical data and relevant variables. Present this data in a clear, organized list along with brief annotations explaining each item's significance. Your output should provide a comprehensive data set for building a formal mathematical representation.
##### FormalTranslation
Transform the extracted data and contextual information into a formal mathematical representation. This may involve setting up equations, inequalities, or expressions that accurately model the problem. Clearly outline the relationships between variables and state any assumptions that are needed for the mathematical model.
##### ChainReasoning
Based on the formal representation, articulate a detailed, step-by-step chain-of-thought that outlines possible solution approaches. Explain each reasoning step, including the mathematical operations and logical inferences you are using to move from the formal model to a potential solution. Your response should create a clear and logical pathway toward solving the problem.
##### PreliminarySolution
Using the chain-of-thought reasoning, propose an initial solution for the math word problem. Provide a detailed sequence of calculations or logical deductions that lead toward a potential answer. Ensure that your approach is well-structured and that every step is clearly justified to facilitate further verification.
##### SolutionVerification
Critically review the preliminary solution provided. Examine each step for mathematical consistency and logical soundness. If the solution appears correct, explicitly state that and explain why. If you find errors or inconsistencies, clearly describe the problems and suggest what needs to be revised. Your feedback will determine the next step in the process.
##### SelfReflection
Reflect on the feedback from the solution verification stage. Analyze any identified errors or weaknesses in the preliminary solution. Critically review your initial approach and propose concrete revisions or alternative strategies to address the issues found. Your insights should pave the way for a more robust solution.
##### AnswerFormulation
Using the refined solution and verification feedback, synthesize your work into a clear final answer. State the final result explicitly and provide a concise, comprehensive explanation of the key reasoning steps that led to this solution. Your final output should be mathematically rigorous and easy to understand.



Saving current state to ../data/2025-04-08_14-05-55/generation=0
Starting crossover and mutation...
Error when calling azure/o3-mini: Error code: 400 - {'error': {'message': "litellm.BadRequestError: litellm.ContentPolicyViolationError: litellm.ContentPolicyViolationError: AzureException - The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766\nmodel=azure/o3-mini. content_policy_fallback=None. fallbacks=None.\n\nSet 'content_policy_fallback' - https://docs.litellm.ai/docs/routing#fallbacks. Received Model Group=azure/o3-mini\nAvailable Model Group Fallbacks=None", 'type': None, 'param': None, 'code': '400'}}

Initializing structured output agent for model AgentOutput (2)
Error while running agent agent-1876d184-6fea-4475-83ea-015d315b1e6d: list index out of range
Crossover agent 

KeyboardInterrupt: 

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [ ]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-2951e239-9fc5-4609-beba-27e50c1d2e3a
## Fitness: 0.2765476293735186
```mermaid 
graph LR
	Start_Node[StartNode] --> Llm1(Problem Understanding)
	Llm1(Problem Understanding) --> Llm2(Problem Analysis)
	Llm2(Problem Analysis) --> Llm3(Solution Generation)
	Llm3(Solution Generation) --> End_Node[EndNode]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models (LLMs) designed to solve math word problems through a structured reasoning process. Each node in the network has a specific role:

1. The Problem Understanding node extracts the key information from the given word problem.
2. The Problem Analysis node evaluates the extracted information to determine the necessary mathematical operations.
3. The Solution Generation node computes the final answer based on the analysis.

Your task is to communicate effectively with the other models in the network, ensuring that your output is clear and informative, as it will guide the next node in the process. Emphasize chain-of-thought reasoning, self-reflection, and evaluative feedback where applicable.
##### Problem Understanding
You are tasked with understanding a math word problem and extracting its key information. Read the provided problem carefully. Identify and list the essential components, including the quantities involved, the relationships between them, and the question that needs to be answered.

Additionally, reflect on the context of the problem and consider if there are any assumptions or clarifications needed. Your output should be a structured summary that clearly delineates the critical elements of the problem, as this will inform the next node in the process.

For example, if the problem states, "John has 5 apples and gives 2 to Mary. How many does he have left?", your output should include:
- Initial quantity of apples: 5
- Quantity given away: 2
- Final question: How many apples are left?
##### Problem Analysis
Your role is to analyze the key information provided by the Problem Understanding node. Based on the structured summary you received, evaluate the relationships between the quantities and determine the mathematical operations needed to solve the problem.

Consider the following:
1. What arithmetic operations (addition, subtraction, multiplication, division) are necessary?
2. Are there multiple steps required to arrive at the solution?
3. Are there any potential pitfalls or common errors that should be noted?

Your output should be a clear, logical outline of the steps and operations required to solve the problem, which will guide the next node in generating the solution. Reflect on your analysis to ensure it is thorough and accurate.
##### Solution Generation
Your task is to generate the final solution to the math word problem based on the analysis provided by the Problem Analysis node. Start by reviewing the steps and operations outlined in the previous node.

Follow these guidelines:
1. Execute the necessary calculations step by step, clearly showing your reasoning.
2. Provide the final answer to the question posed in the word problem.
3. Include a brief reflection on the solution process, highlighting any assumptions made or potential errors to watch out for in similar problems.

Your output should be the computed answer and a concise explanation of how you arrived at it, ensuring that the final result is clear and informative for the EndNode.

